In [1]:
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline
import time

In [2]:
#to keep things in order, and to avoid to copy and paste everytime our functions if we want to use them in more than one folder,
#we can temporarily use this library. 
import sys

#in this way Python will search the implementations also in the path '../HelperFunctions'
sys.path.insert(0, '../HelperFunctions')
sys.path.insert(0, '../pre-processing/Clean_Data/')

In [3]:
from proj1_helpers import *
from common_functions import *
from counters import *
from replace import *
from regressors import batch_iter

In [4]:
yb, input_data, ids = load_csv_data("../data/train.csv", sub_sample=False)

In [5]:
#this will surely be deleted, in this way we are sure that original_data is the original version of the data and we don't have
#to load them again
from copy import deepcopy
originalData = deepcopy(input_data)
originalY = deepcopy(yb)

# Functions from lab 5

In [6]:
# def sig(t):
#     """apply sigmoid function on t."""
#     if t > 0:
#         return 1 / (1 + np.exp(-t))
#     else:
#         return np.exp(t) / (1 + np.exp(t))

# sigmoid = np.vectorize(sig)

def sigmoid(t):
     return .5*(1+np.tanh(.5 * t))

def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    pred = sigmoid(tx.dot(w))
    correctZero = 1e-7
    loss = np.sum((1 - y) * np.log(1 - pred + correctZero) + y * np.log(pred + correctZero))
    #return np.squeeze(- loss)
    return -loss/y.shape[0]


def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    pred = sigmoid(tx.dot(w))
    grad = tx.T.dot(pred - y)
    return grad/y.shape[0]

def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descen using logistic regression.
    Return the loss and the updated w.
    """
    #batch = next(batch_iter(y, tx, 32))
    #minibatch_y, minibatch_tx = batch[0], batch[1]
    grad = calculate_gradient(y, tx, w)
    loss = calculate_loss(y, tx, w)
    w = w - gamma * grad
    return loss, w

In [42]:
#Creation of tx

input_data = deepcopy(originalData)
print(input_data.shape)
#Clean the dataset
numInvalidValues = countInvalid(input_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
input_data = replaceWithZero(input_data,-999,idxCols)

#standardize
tx,_,_ = standardize(input_data)

#add ones
tx = np.c_[np.ones((yb.shape[0], 1)), tx]
y = yb

(250000, 30)


In [45]:
#first tries with gradient descent

Y, TX = sample_data(y, tx, 10, 150000)
max_iter = 10000
w = np.zeros(tx.shape[1])
gamma = 0.0005
loss0 = 0
print(TX.shape)

for iter in range(max_iter):
    wold = w
    loss, w = learning_by_gradient_descent(Y, TX, w, gamma)

    if iter%100==0:
        wnew = w
        print(f'Iter = {iter}, Loss = {loss}, |wold - wnew| = {np.linalg.norm(wold-wnew)}')




(150000, 31)
Iter = 0, Loss = 0.6931469805599657, |wold - wnew| = 0.0005621292061665366
Iter = 100, Loss = 0.6322868630951723, |wold - wnew| = 0.0005413858962153733
Iter = 200, Loss = 0.5756629852485209, |wold - wnew| = 0.000523026864314041
Iter = 300, Loss = 0.5226572709361063, |wold - wnew| = 0.0005068003548763514
Iter = 400, Loss = 0.47275225673990545, |wold - wnew| = 0.0004924296821479786
Iter = 500, Loss = 0.42552008039490075, |wold - wnew| = 0.00047965102095496437
Iter = 600, Loss = 0.3806083546999344, |wold - wnew| = 0.00046822983664032133
Iter = 700, Loss = 0.3377264620444261, |wold - wnew| = 0.00045796506538703645
Iter = 800, Loss = 0.2966336651441241, |wold - wnew| = 0.0004486874923474239
Iter = 900, Loss = 0.257129326432646, |wold - wnew| = 0.00044025591382528657
Iter = 1000, Loss = 0.21904506695911646, |wold - wnew| = 0.00043255283034905167
Iter = 1100, Loss = 0.18223855118655638, |wold - wnew| = 0.0004254804258357327
Iter = 1200, Loss = 0.14658857614177684, |wold - wnew| =

KeyboardInterrupt: 

In [ ]:
tx

Let's try to generate a submission

In [ ]:
WWW = w 

In [ ]:
xxx, test_data, ids = load_csv_data("../data/test.csv", sub_sample=False)

In [ ]:
#we need to preprocess the test_data as we preprocessed the train data

numInvalidValues = countInvalid(test_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
test_data = replaceWithZero(test_data,-999,idxCols)
txTest,_,_ = standardize(test_data)
txTest = np.c_[np.ones((xxx.shape[0], 1)), txTest]

In [ ]:
y_pred = predict_labels(WWW, txTest)

In [ ]:
create_csv_submission(ids, y_pred, 'submission.csv')

# Same things with PCA

In [26]:
sys.path.insert(0, '../pre-processing/PCA/')

In [46]:
from pca_functions import PCAWithCovariance

input_data = deepcopy(originalData)
y = deepcopy(originalY)
print(input_data.shape)
print(y.shape)
numInvalidValues = countInvalid(input_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
input_data = replaceWithZero(input_data,-999,idxCols)
input_data,_,_ = standardize(input_data)

(250000, 30)
(250000,)


In [72]:
#Generating the principal components

_,eV = PCAWithCovariance(input_data)

N = 2 #num p. components
components = np.empty(input_data.shape[0])
for i in range(N):
    components = np.c_[components, input_data.dot(eV[:,i])]
    
#tx = np.c_[np.ones(input_data.shape[0]), components]
tx = components

In [73]:
Y, TX = sample_data(y, tx, 1, 70000)
TX.shape

(70000, 3)

In [74]:
max_iter = 10000
w = np.zeros(TX.shape[1])
gamma = 0.005
loss0 = 0
for iter in range(max_iter):
    wold = w
    loss, w = learning_by_gradient_descent(Y, TX, w, gamma)

    if iter%500==0:
        wnew = w
        print(f'Iter = {iter}, Loss = {loss}, |wold - wnew| = {np.linalg.norm(wold-wnew)}')

Iter = 0, Loss = 0.6931469805599655, |wold - wnew| = 0.0031179742471667757
Iter = 500, Loss = 0.5447375044285359, |wold - wnew| = 0.0004555830001966787
Iter = 1000, Loss = 0.5342454037057077, |wold - wnew| = 0.0002189034962146976
Iter = 1500, Loss = 0.5314790629170908, |wold - wnew| = 0.00012074029204930974
Iter = 2000, Loss = 0.5305906522236288, |wold - wnew| = 7.059816906828605e-05
Iter = 2500, Loss = 0.5302781788975787, |wold - wnew| = 4.256537382383867e-05
Iter = 3000, Loss = 0.5301627597796059, |wold - wnew| = 2.6113614835430183e-05
Iter = 3500, Loss = 0.5301189127039768, |wold - wnew| = 1.6185892921585408e-05
Iter = 4000, Loss = 0.5301019756231156, |wold - wnew| = 1.0095015865962806e-05
Iter = 4500, Loss = 0.5300953673352786, |wold - wnew| = 6.320300985399138e-06
Iter = 5000, Loss = 0.5300927735057194, |wold - wnew| = 3.9664210696567164e-06
Iter = 5500, Loss = 0.5300917519048081, |wold - wnew| = 2.492888464328479e-06
Iter = 6000, Loss = 0.5300913488693026, |wold - wnew| = 1.56822

In [ ]:
# generate submission with PCA
xxx, test_data, ids = load_csv_data("../data/test.csv", sub_sample=False)



In [ ]:
numInvalidValues = countInvalid(test_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
test_data = replaceWithZero(test_data,-999,idxCols)
test_data,_,_ = standardize(test_data)

components = np.empty(test_data.shape[0])
for i in range(N):
    components = np.c_[components, test_data.dot(eV[:,i])]
    
#tx_test = np.c_[np.ones(test_data.shape[0]), components]
tx_test = components

y_pred = predict_labels(w, tx_test)
create_csv_submission(ids, y_pred, 'submission.csv')